## Mulearn optimization problem test

In [1]:
import kernel
import numpy as np
import cooper
import torch
import time
import copy
import random
from importlib import reload

In [2]:
import __init__
import distributions
import kernel
import fuzzifier
import optimization

reload(__init__)
reload(distributions)
reload(kernel)
reload(fuzzifier)
reload(optimization)

from __init__ import *
from distributions import *
from kernel import *
from fuzzifier import *
from optimization import *

tensorflow not available
tensorflow not available


In [27]:
torch.set_default_device('cuda:0')

In [8]:
torch.set_default_device('cpu')

### $\chi$ version

$max \sum_i{\chi_ik_{ii} - \sum_{i,j}\chi_i\chi_jk_{ij}}$

where $k_{ij} = k(i,j)$, under constraints:

$\sum_i \chi_i = 1$

$-C(1 - \mu_i) \le \chi_i \le C \mu_i$

In [65]:
class mulearnCMP(cooper.ConstrainedMinimizationProblem):
    def __init__(self):
        super().__init__(is_constrained=True)

    def closure(self, k, mus, C, chis):

        loss = -chis.dot(torch.diag(k,0)) + chis.dot(k.matmul(chis)) 

        eq_defect = torch.sum(chis) - 1

        ineq_defect1 = -C*(1 - mus) - chis
        ineq_defect2 = chis - C*mus

        ineq_defect = torch.stack([ineq_defect1, ineq_defect2])
        
        return cooper.CMPState(loss=loss, ineq_defect=ineq_defect, eq_defect=eq_defect)

In [72]:
def train(k, mus, C, max_iter, lr, atol):

    mus = torch.tensor(mus, dtype=torch.float64)

    if type(k) != torch.Tensor:
        k = torch.tensor(k, dtype=torch.float64)

    cmp = mulearnCMP()
    formulation = cooper.LagrangianFormulation(cmp)

    #np.random.seed(42)
    a = np.random.uniform(-0.1, 0.1, len(mus))
    #a = np.zeros(len(mus))
    
    chis = torch.nn.Parameter(torch.tensor(a, dtype=torch.float64, requires_grad=True))

    primal_optimizer = cooper.optim.ExtraAdam([chis], lr=lr)
    dual_optimizer = cooper.optim.partial_optimizer(cooper.optim.ExtraAdam, lr=lr)
    
    constrained_optimizer = cooper.ConstrainedOptimizer(
        formulation=formulation,
        primal_optimizer=primal_optimizer,
        dual_optimizer=dual_optimizer,
    )

    const1 = torch.tensor(1.0, dtype=torch.float64)
    
    gap = torch.tensor(torch.inf)
    constr1, constr2, constr3 = False, False, False
    opt_found = False
    
    i = 0

    while (i < max_iter):
        
        constrained_optimizer.zero_grad()
        
        lagrangian = formulation.composite_objective(
            cmp.closure, k, mus, C, chis
        )
        formulation.custom_backward(lagrangian)
        
        constrained_optimizer.step(cmp.closure, k, mus, C, chis)
        
        constr1 = almost_negative(formulation.cmp.state.ineq_defect[0], atol=atol)
        constr2 = almost_negative(formulation.cmp.state.ineq_defect[1], atol=atol)
        constr3 = torch.isclose(torch.sum(chis), const1, atol=atol)

        gap = formulation.cmp.state.loss - formulation.dual_parameters[1]

        
        #print('i:', i, 'gap:', gap)
        #print(constr1, constr2, constr3)

        if (torch.abs(gap) <= 1e-2) and constr1 and constr2 and constr3:
            opt_found = True
            break
        
        i+=1


    if not opt_found:
        print('optimal values not found')
    
    return chis#optim_chis, minimum_loss

        
def generate_dataset(n_samples=10, n_features=1):

    #random.seed(42)
    
    X = np.zeros((n_samples, n_features))

    for elem in range(n_samples):
        for dim in range(n_features):
            X[elem][dim] = random.uniform(0,1)

    gaussian = lambda x,var : (np.e ** (-(((x-0.5)*2)**2)/(2*var)))

    y = np.zeros(n_samples)
    
    for elem in range(n_samples):
        y[elem] = 1
        for dim in range(n_features):
            y[elem] *= gaussian(X[elem][dim], np.var([X[j][dim] for j in range(n_samples)]))
                          
    return X,y

def almost_negative(tensor, atol):
    return torch.isclose(tensor[torch.gt(tensor,0.0)], torch.tensor(0.0, dtype=torch.float64), atol=atol).all()

In [33]:
X, mus = generate_dataset(500)

In [47]:
kern = kernel.GaussianKernel()
s = time.time()
k = [[kern.compute(x1, x2) for x1 in X] for x2 in X]
print('time:', time.time() - s)

time: 0.9381577968597412


In [58]:
C = 1

torch.set_default_device('cpu')

s = time.time()
chis = train(k, mus, C, 5000, 1e-3, 0.01)
#chis = train(k, mus, C)
print('time:', time.time() - s)

time: 1.6869761943817139


In [48]:
C = 1

torch.set_default_device('cuda:0')

s = time.time()
chis, loss = train(k, mus, C, 1000, 1e-3, 1e-3)
#chis = train(k, mus, C)
print('time:', time.time() - s)

optim not found
time: 5.458599805831909


### $\alpha$ and $\beta$ version

$max \sum_i{(\alpha_i \mu_i - \beta_i(1 - \mu_i))k_{ii} - \sum_{i,j}(\alpha_i \mu_i - \beta_i(1 - \mu_i))(\alpha_j \mu_j - \beta_j(1 - \mu_j))k_{ij}}$

where $k_{ij} = k(i,j)$, under constraints:

$\sum_i \alpha_i \mu_i - \beta_i(1 - \mu_i) = 1$

$0 \le \alpha_i,\beta_i \le C$

In [22]:
class mulearnCMP2(cooper.ConstrainedMinimizationProblem):
    def __init__(self):
        super().__init__(is_constrained=True)

    def closure(self, k, mus, C, alphas, betas):

        chis = alphas*mus + betas*(1-mus)

        #print(chis)
        
        loss = -chis.dot(torch.diag(k,0)) + chis.dot(k.matmul(chis)) 

        eq_defect = torch.sum(chis) - 1

        ineq_defect1 = alphas - C
        ineq_defect2 = -alphas
        ineq_defect3 = betas - C
        ineq_defect4 = -betas
        
        ineq_defect = torch.stack([ineq_defect1, ineq_defect2, ineq_defect3, ineq_defect4])
        
        return cooper.CMPState(loss=loss, ineq_defect=ineq_defect, eq_defect=eq_defect)

In [26]:
def train2(k, mus, C, max_iter, lr, atol):

    mus = torch.tensor(mus, dtype=torch.float64)

    k = torch.tensor(k, dtype=torch.float64)

    cmp = mulearnCMP2()
    formulation = cooper.LagrangianFormulation(cmp)

    #np.random.seed(42)
    a = np.random.uniform(-0.1, 0.1, len(mus))
    b = np.random.uniform(-0.1, 0.1, len(mus))
    
    alphas = torch.nn.Parameter(torch.tensor(a, dtype=torch.float64, requires_grad=True))
    betas = torch.nn.Parameter(torch.tensor(b, dtype=torch.float64, requires_grad=True))
    
    primal_optimizer = cooper.optim.ExtraAdam([alphas, betas], lr=lr)
    dual_optimizer = cooper.optim.partial_optimizer(cooper.optim.ExtraAdam, lr=lr)
    
    constrained_optimizer = cooper.ConstrainedOptimizer(
        formulation=formulation,
        primal_optimizer=primal_optimizer,
        dual_optimizer=dual_optimizer,
    )

    const1 = torch.tensor(0.0, dtype=torch.float64)
    
    gap = torch.tensor(torch.inf)
    constr1, constr2, constr3, constr4, constr5= False, False, False, False, False
    opt_found = False
    
    i = 0

    while (i < max_iter):
        
        constrained_optimizer.zero_grad()
        
        lagrangian = formulation.composite_objective(
            cmp.closure, k, mus, C, alphas, betas
        )
        formulation.custom_backward(lagrangian)
        
        constrained_optimizer.step(cmp.closure, k, mus, C, alphas, betas)
        
        constr1 = almost_negative(formulation.cmp.state.ineq_defect[0], atol=atol)
        constr2 = almost_negative(formulation.cmp.state.ineq_defect[1], atol=atol)
        constr3 = almost_negative(formulation.cmp.state.ineq_defect[2], atol=atol)
        constr4 = almost_negative(formulation.cmp.state.ineq_defect[3], atol=atol)
        constr5 = torch.isclose(formulation.cmp.state.eq_defect, const1, atol=atol)

        gap = formulation.cmp.state.loss - formulation.dual_parameters[1]
        
        #print('i:', i, 'gap:', gap)
        #print(constr1, constr2, constr3, constr4, constr5)

        if (torch.abs(gap) <= 1e-2) and constr1 and constr2 and constr3 and constr4 and constr5:
            opt_found = True
            break
        
        i+=1


    if not opt_found:
        print('optimal values not found')

    
    return alphas*mus - betas*(1-mus)

        
def generate_dataset(n_samples=10, n_features=1):

    #random.seed(42)
    
    X = np.zeros((n_samples, n_features))

    for elem in range(n_samples):
        for dim in range(n_features):
            X[elem][dim] = random.uniform(0,1)

    gaussian = lambda x,var : (np.e ** (-(((x-0.5)*2)**2)/(2*var)))

    y = np.zeros(n_samples)
    
    for elem in range(n_samples):
        y[elem] = 1
        for dim in range(n_features):
            y[elem] *= gaussian(X[elem][dim], np.var([X[j][dim] for j in range(n_samples)]))
                          
    return X, y

def almost_negative(tensor, atol):
    return torch.isclose(tensor[torch.gt(tensor,0.0)], torch.tensor(0.0, dtype=torch.float64), atol=atol).all()

In [27]:
X, mus = generate_dataset(500)

In [28]:
kern = kernel.GaussianKernel()
s = time.time()
k = [[kern.compute(x1, x2) for x1 in X] for x2 in X]
print('time:', time.time() - s)

time: 0.9422235488891602


In [47]:
C = 1

torch.set_default_device('cpu')

s = time.time()
chis = train2(k, mus, C, 5000, 1e-3, 0.01)

print('time:', time.time() - s)

optimal values not found
time: 10.48835301399231
